In [1]:
using Pkg
Pkg.activate(".")

  Activating environment at `~/Orbit_Interactive_Plots/Project.toml`


In [2]:
#Fausto Vega 

#Interactive Plot to show covariance results of different satellite configurations using 
#Time of Arrival and the Doppler covariance method

using GLMakie, FileIO, Colors
using LinearAlgebra
using BenchmarkTools
using StaticArrays
using SatelliteDynamics
using DelimitedFiles
using ForwardDiff

┌ Info: Precompiling GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a]
└ @ Base loading.jl:1342


In [3]:
const G = 6.67430e-20 #km**3/(kg * s**2)
const m_1 = 5.97219e24 #mass of the Earth in kg
const m_2 = 1 #mass of satellite in kg
const μ = G*m_1 #gravitational parameter
const R_earth = 6378.12 # radius of Earth in km
h = 1 #time step in seconds

1

In [4]:
#TOA scales
distance_scale = R_EARTH*(1e-3) # scales km to custom scale
time_scale = 1/(C_LIGHT/R_EARTH) #scales seconds to custom scale
c = 1 # speed of light

#Doppler scales
distance_scale_doppler = R_EARTH*(1e-3) #scales km to custom scale
time_scale_doppler = (1/(C_LIGHT/R_EARTH))*1e4 #scales seconds to custom scale
velocity_scale_doppler = distance_scale_doppler/time_scale_doppler #scales km/s to custom units
c_doppler = 1*1e4 #speed of light

10000.0

In [5]:
#Find the zenith angle

function elevation_angle(satposes, r0)
    
    elevangle = zeros(4)

    #vector between tag and the satellite
    normalvec = r0
    
    for i in 1:4
        
        vector = satposes[i,:] - r0[1:3]
        
        #Find the angle between the normal and the vector going from the tag to the satellte
        theta = acos(dot(normalvec,vector)/(norm(normalvec)*norm(vector))) * (180/pi)
        
        #save elevation angle
        elevangle[i] = theta
    end
    
    return elevangle
end

elevation_angle (generic function with 1 method)

In [6]:
#satellite dynamics 

function sat_dynamics(x)
    
    sat_pose = SA[x[1], x[2], x[3]]
    
    #This WORKS
    upperleft = @SMatrix zeros(3,3)
    upperright = SMatrix{3,3}(1.0I)
    lowerleft = SMatrix{3,3}(1I)*(-μ/(norm(sat_pose))^3)
    lowerright = @SMatrix zeros(3,3)
    
    
    upper = [upperleft upperright]
    lower = [lowerleft lowerright]
    
    A = [upper; lower]
    
    xdot = A*x
    
    return xdot
    
end

sat_dynamics (generic function with 1 method)

In [7]:
#Runga Kutta 4th order algorithm to find the next state

function RK4_orbit(x, h)
    
    f1_ = sat_dynamics(x)
    f2_ = sat_dynamics(x+0.5*h*f1_)
    f3_ = sat_dynamics(x+0.5*h*f2_)
    f4_ = sat_dynamics(x+h*f3_)
    
    xnext = x+(h/6.0)*(f1_+2*f2_+2*f3_+f4_)

    return xnext
    
end

RK4_orbit (generic function with 1 method)

In [8]:
#Propagate the orbit

function twobody_rk4(fun, x0, Tf, h)
    
    t = Array(range(0,Tf, step=h)) #create a range to final time at constant time step
    
    all_x = zeros(length(x0), length(t)) #variable to store all x
    
    all_x[:,1] .= x0 #set the initial state
    
    for k=1:(length(t) - 1)
        all_x[:,k+1] .= RK4_orbit(all_x[:,k], h)
    end
    
    return all_x, t
end

twobody_rk4 (generic function with 1 method)

In [9]:
#get the tag xyz and the initial guesses

function tag_xyz(latitude)

    #Setting a known tag position
    #Fix the longitude and altitude, vary the latitude

    #longitude, latitude, altitude
    tag_geof = [-165.4545, latitude, 0]

    #gives tag position in meters
    tag = sGEOCtoECEF(tag_geof, use_degrees = true)

    #Equitorial Position
    #Rescale to km
    x0= tag[1]*1e-3
    y0 = tag[2]*1e-3
    z0 = tag[3]*1e-3
    t0 = 1e-5
    bdot = 1e4*1e-3 #convert to km/s almost working

    #TOA unknowns unscaled 
    r0 = [x0, y0, z0, t0]

    #TOA unknowns scaled
    r0_scaled = [x0/distance_scale, y0/distance_scale, z0/distance_scale, t0/time_scale]

    #knowns for Doppler case
    TEC1 = 3e-4
    TEC2 = 4e-4
    TEC3 = 3.5e-4
    TEC4 = 4.5e-4

    #Doppler unknowns scaled
    r0_scaled_Doppler = [x0/distance_scale_doppler, y0/distance_scale_doppler, z0/distance_scale_doppler, bdot/velocity_scale_doppler, TEC1, TEC2, TEC3, TEC4]
    
    
    return r0, r0_scaled, r0_scaled_Doppler
end

tag_xyz (generic function with 1 method)

In [10]:
#update the orbit when the slider changes

function orbit_update(trueanom, RAAN_sep, delta_sep, altitude)
    
    iss1 = [6371e3 + (altitude*1e3), 0.0004879, 90.6391, 194.0859- (RAAN_sep/2), 151.2014, 190];
    iss2 = [6371e3 + (altitude*1e3), 0.0004879, 90.6391, 194.0859 - (RAAN_sep/2), 151.2014, 190+trueanom];
    iss3 = [6371e3 + (altitude*1e3), 0.0004879, 90.6391, 195.0859 + (RAAN_sep/2), 151.2014, 190+delta_sep]; 
    iss4 = [6371e3 + (altitude*1e3), 0.0004879, 90.6391, 195.0859 + (RAAN_sep/2), 151.2014, 190+delta_sep+trueanom]; 
    
    eci0_1 = sOSCtoCART(iss1, use_degrees=true)
    eci0_2 = sOSCtoCART(iss2, use_degrees=true)
    eci0_3 = sOSCtoCART(iss3, use_degrees=true)
    eci0_4 = sOSCtoCART(iss4, use_degrees=true)
    
    T = orbit_period(iss1[1])
    
    #in km
    x0_1 = SA[eci0_1[1], eci0_1[2], eci0_1[3],eci0_1[4], eci0_1[5], eci0_1[6]]*1e-3 
    x0_2 = SA[eci0_2[1], eci0_2[2], eci0_2[3],eci0_2[4], eci0_2[5], eci0_2[6]]*1e-3 
    x0_3 = SA[eci0_3[1], eci0_3[2], eci0_3[3],eci0_3[4], eci0_3[5], eci0_3[6]]*1e-3 
    x0_4 = SA[eci0_4[1], eci0_4[2], eci0_4[3],eci0_4[4], eci0_4[5], eci0_4[6]]*1e-3 
    
    #Find state of orbit 1,2,3,4
    eci_1, t_hist1 = twobody_rk4(sat_dynamics, x0_1, T, h)
    eci_2, t_hist2 = twobody_rk4(sat_dynamics, x0_2, T, h)
    eci_3, t_hist3 = twobody_rk4(sat_dynamics, x0_3, T, h)
    eci_4, t_hist4 = twobody_rk4(sat_dynamics, x0_4, T, h)
    
    #6557 is the size of the array when it is at 1200. Need to make it a fixed size to be able to make it 
    #interactive
    
    size_eci1 = size(eci_1)
    size_eci2 = size(eci_2)
    size_eci3 = size(eci_3)
    size_eci4 = size(eci_4)
    
    eci_1_sized = zeros(6,6557)
    eci_2_sized = zeros(6,6557)
    eci_3_sized = zeros(6,6557)
    eci_4_sized = zeros(6,6557)
    
    eci_1_sized[1:size_eci1[1], 1:size_eci1[2]] = eci_1
    eci_2_sized[1:size_eci2[1], 1:size_eci2[2]] = eci_2
    eci_3_sized[1:size_eci3[1], 1:size_eci3[2]] = eci_3
    eci_4_sized[1:size_eci4[1], 1:size_eci4[2]] = eci_4
    
    centroid_guess = [(eci_1[1,1]+eci_2[1,1]+eci_3[1,1]+eci_4[1,1])/4, (eci_1[2,1]+eci_2[2,1]+eci_3[2,1]+eci_4[2,1])/4, (eci_1[3,1]+eci_2[3,1]+eci_3[3,1]+eci_4[3,1])/4] 
    onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
    geodetic = [onearth[1], onearth[2], 0]

    #Guess
    xyz = sGEOCtoECEF(geodetic, use_degrees = true)*1e-3
    
    return eci_1_sized, eci_2_sized, eci_3_sized, eci_4_sized, xyz
        
end

orbit_update (generic function with 1 method)

In [11]:
#Start the simulation at this configuration

trueanom = 10
RAAN_sep = 2
delta_sep = 3
altitude = 1200

#initial
eci_1_s, eci_2_s, eci_3_s, eci_4_s, guess_s = orbit_update(trueanom, RAAN_sep, delta_sep, altitude);

In [12]:
#create the covariance matrix for
using BlockDiagonals
#pass in time and frequency accuracy unscaled
function get_P(time_accuracy, frequency_accuracy)
    position_accuracy = 0.1 * 1e-3/distance_scale #(0.1 meters) 2 sigma standard deviation
    velocity_accuracy = 0.01 *1e-3/velocity_scale_doppler #(0.01 meters/second) 2 sigma standard deviation
    
    #covariance matrix is made up by the squares of the standard deviation (variance)
    P_TOA_1sat = diagm([(position_accuracy)^2, (position_accuracy)^2, (position_accuracy)^2, (time_accuracy/time_scale)^2])
    P_Doppler_1sat = diagm([(position_accuracy)^2, (position_accuracy)^2, (position_accuracy)^2, (velocity_accuracy)^2, (velocity_accuracy)^2, (velocity_accuracy)^2, (frequency_accuracy)^2])
    
    #Make the block diagonal for the 8 measurments
    
    P_TOA = BlockDiagonal([P_TOA_1sat, P_TOA_1sat, P_TOA_1sat, P_TOA_1sat, P_TOA_1sat, P_TOA_1sat, P_TOA_1sat, P_TOA_1sat])
    P_Doppler = BlockDiagonal([P_Doppler_1sat , P_Doppler_1sat , P_Doppler_1sat , P_Doppler_1sat , P_Doppler_1sat , P_Doppler_1sat , P_Doppler_1sat , P_Doppler_1sat ])

    return P_TOA, P_Doppler
end

get_P (generic function with 1 method)

In [13]:
# time_accuracy = 1e-8 
# frequency_accuracy = 1 

# P_TOA_estimate, P_Doppler_estimate = get_P(time_accuracy, frequency_accuracy)

In [14]:
#this is the covariance of the satellite positions subject to the noise given 
#Pin_read_Doppler = readdlm("sat_cov_Doppler_2freq.txt")

In [15]:
#Pin_read_TOA = readdlm("sat_cov_TOA_2freq.txt")

In [16]:
#For Time of Arrival Case
function measurment_residual(r0, satposes,zenith, t, f2)
    
    #rotation of the Earth
    omega = OMEGA_EARTH*time_scale
    
    #scale to improve conditioning. 
    #used to include the ionosphere effects
    scale_f1 = (40.3/(f1)^2)*1e22
    scale_f2 = (40.3/(f2)^2)*1e22
    
    angles = zeros(4)
    
    for i=1:4
        angles[i] = asind((R_EARTH*sind(zenith[i]))/(R_EARTH + hi))
    end
    
    #Find rotation matrices to include the rotation of the Earth
    A1 = [cos(omega*t[1]) sin(omega*t[1]) 0;
          -sin(omega*t[1]) cos(omega*t[1]) 0;
          0 0 1];
    A2 = [cos(omega*t[2]) sin(omega*t[2]) 0;
          -sin(omega*t[2]) cos(omega*t[2]) 0;
          0 0 1];
    A3 = [cos(omega*t[3]) sin(omega*t[3]) 0;
          -sin(omega*t[3]) cos(omega*t[3]) 0;
          0 0 1];
    A4 = [cos(omega*t[4]) sin(omega*t[4]) 0;
          -sin(omega*t[4]) cos(omega*t[4]) 0;
          0 0 1];
    A5 = [cos(omega*t[5]) sin(omega*t[5]) 0;
          -sin(omega*t[5]) cos(omega*t[5]) 0;
          0 0 1];
    A6 = [cos(omega*t[6]) sin(omega*t[6]) 0;
          -sin(omega*t[6]) cos(omega*t[6]) 0;
          0 0 1];
    A7 = [cos(omega*t[7]) sin(omega*t[7]) 0;
          -sin(omega*t[7]) cos(omega*t[7]) 0;
          0 0 1];
    A8 = [cos(omega*t[8]) sin(omega*t[8]) 0;
          -sin(omega*t[8]) cos(omega*t[8]) 0;
          0 0 1];
    
    #residuals for time measurment residual
    
    res1 = norm(r0[1:3] - A1*satposes[1,1:3]) - c*(t[1] - r0[4]) + (scale_f1*(r0[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res2 = norm(r0[1:3] - A2*satposes[2,1:3]) - c*(t[2] - r0[4]) + (scale_f1*(r0[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res3 = norm(r0[1:3] - A3*satposes[3,1:3]) - c*(t[3] - r0[4]) + (scale_f1*(r0[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res4 = norm(r0[1:3] - A4*satposes[4,1:3]) - c*(t[4] - r0[4]) + (scale_f1*(r0[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    res5 = norm(r0[1:3] - A5*satposes[1,1:3]) - c*(t[5] - r0[4]) + (scale_f2*(r0[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res6 = norm(r0[1:3] - A6*satposes[2,1:3]) - c*(t[6] - r0[4]) + (scale_f2*(r0[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res7 = norm(r0[1:3] - A7*satposes[3,1:3]) - c*(t[7] - r0[4]) + (scale_f2*(r0[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res8 = norm(r0[1:3] - A8*satposes[4,1:3]) - c*(t[8] - r0[4]) + (scale_f2*(r0[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    
    return [res1, res2, res3, res4, res5, res6, res7, res8]
    
end

measurment_residual (generic function with 1 method)

In [17]:
#For Time of Arrival Case

function residual(x, sat_pose_t, zenith, f2)
    
    #rotation of the Earth 
    omega = OMEGA_EARTH*time_scale
    
    angles = zeros(4)
    
    for i=1:4
        angles[i] = asind((R_EARTH*sind(zenith[i]))/(R_EARTH + hi))
    end

    scale_f1 = (40.3/(f1)^2)*1e22
    scale_f2 = (40.3/(f2)^2)*1e22
    
    A1 = [cos(omega*sat_pose_t[1,4]) sin(omega*sat_pose_t[1,4]) 0;
          -sin(omega*sat_pose_t[1,4]) cos(omega*sat_pose_t[1,4]) 0;
          0 0 1];
    A2 = [cos(omega*sat_pose_t[2,4]) sin(omega*sat_pose_t[2,4]) 0;
          -sin(omega*sat_pose_t[2,4]) cos(omega*sat_pose_t[2,4]) 0;
          0 0 1];
    A3 = [cos(omega*sat_pose_t[3,4]) sin(omega*sat_pose_t[3,4]) 0;
          -sin(omega*sat_pose_t[3,4]) cos(omega*sat_pose_t[3,4]) 0;
          0 0 1];
    A4 = [cos(omega*sat_pose_t[4,4]) sin(omega*sat_pose_t[4,4]) 0;
          -sin(omega*sat_pose_t[4,4]) cos(omega*sat_pose_t[4,4]) 0;
          0 0 1];
    A5 = [cos(omega*sat_pose_t[5,4]) sin(omega*sat_pose_t[5,4]) 0;
          -sin(omega*sat_pose_t[5,4]) cos(omega*sat_pose_t[5,4]) 0;
          0 0 1];
    A6 = [cos(omega*sat_pose_t[6,4]) sin(omega*sat_pose_t[6,4]) 0;
          -sin(omega*sat_pose_t[6,4]) cos(omega*sat_pose_t[6,4]) 0;
          0 0 1];
    A7 = [cos(omega*sat_pose_t[7,4]) sin(omega*sat_pose_t[7,4]) 0;
          -sin(omega*sat_pose_t[7,4]) cos(omega*sat_pose_t[7,4]) 0;
          0 0 1];
    A8 = [cos(omega*sat_pose_t[8,4]) sin(omega*sat_pose_t[8,4]) 0;
          -sin(omega*sat_pose_t[8,4]) cos(omega*sat_pose_t[8,4]) 0;
          0 0 1];
    
    res1 = norm(x[1:3] - A1*sat_pose_t[1,1:3]) - c*(sat_pose_t[1,4] - x[4]) + (scale_f1*(x[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res2 = norm(x[1:3] - A2*sat_pose_t[2,1:3]) - c*(sat_pose_t[2,4] - x[4]) + (scale_f1*(x[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res3 = norm(x[1:3] - A3*sat_pose_t[3,1:3]) - c*(sat_pose_t[3,4] - x[4]) + (scale_f1*(x[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res4 = norm(x[1:3] - A4*sat_pose_t[4,1:3]) - c*(sat_pose_t[4,4] - x[4]) + (scale_f1*(x[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    res5 = norm(x[1:3] - A5*sat_pose_t[1,1:3]) - c*(sat_pose_t[5,4] - x[4]) + (scale_f2*(x[5]/cosd(angles[1])))*(1e-3)/distance_scale;
    res6 = norm(x[1:3] - A6*sat_pose_t[2,1:3]) - c*(sat_pose_t[6,4] - x[4]) + (scale_f2*(x[6]/cosd(angles[2])))*(1e-3)/distance_scale;
    res7 = norm(x[1:3] - A7*sat_pose_t[3,1:3]) - c*(sat_pose_t[7,4] - x[4]) + (scale_f2*(x[7]/cosd(angles[3])))*(1e-3)/distance_scale;
    res8 = norm(x[1:3] - A8*sat_pose_t[4,1:3]) - c*(sat_pose_t[8,4] - x[4]) + (scale_f2*(x[8]/cosd(angles[4])))*(1e-3)/distance_scale;
    
    return [res1, res2, res3, res4, res5, res6, res7, res8]
    
end

residual (generic function with 1 method)

In [18]:
#Function to obtain time measurments (truth values)

function get_time(r0_TEC, satposes, zenith, f2)
    
    n = 1000

    time = [zeros(8) for i = 1:1000]
    Rt = [zeros(8) for i = 1:1000]

    #initial guess
    time[1] = [0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale, 0.1\time_scale] #initial guess

    Rt[1] = measurment_residual(r0_TEC, satposes, zenith, time[1], f2)

    iters = 0

    for i=1:n

        Rt[i] = measurment_residual(r0_TEC, satposes, zenith, time[i], f2)

        #println("this is norm of residual: ", norm(Rt[i]))
        
        iters += 1

        if(norm(Rt[i]) < 1e-6)
            break
        end

        jacobian = ForwardDiff.jacobian(dt -> measurment_residual(r0_TEC, satposes, zenith, dt, f2), time[i])

        deltat = (jacobian)\-Rt[i]

        time[i+1]  = time[i] + deltat
    end
    
    Rt = Rt[1:iters]
    time = time[1:iters]
    time_measurment = time[end]
    
    return time_measurment
    
end

get_time (generic function with 1 method)

In [19]:
#frequency variables
f1 = 400e6 #in Hz


#f2 = 600e6  #in Hz

4.0e8

In [20]:
#TOA Jacobian for Least Squares Estimation

function LS_Amatrix(x, y, scale, A, angle)
    
    jacobian_pose = (x[1:3] - A*y[1:3])/norm(x[1:3]-A*y[1:3])
    
    dpdtau = c
    dpdTEC = (scale/cosd(angle))*(1e-3)/distance_scale
    
    return [jacobian_pose; dpdtau; dpdTEC]
end

LS_Amatrix (generic function with 1 method)

In [21]:
#TOA Jacobian for Least Squares Estimation

function LS_Amatrix_dpdy(x, y, scale, A, angle)
    
    #c = C_LIGHT
    
    #works
    jacobian_pose = (y[1:3] - A'*x[1:3])/norm(x[1:3]-A*y[1:3])
    
    dpdtau = -c
    
    return [jacobian_pose; dpdtau]
end

LS_Amatrix_dpdy (generic function with 1 method)

In [22]:
function customjacobian(initial_x0,sat_pose_t, elev_angles, f2)
    c=1
    
    while norm(residual(initial_x0, sat_pose_t, elev_angles, f2))>1e-10
        
        omega = OMEGA_EARTH*time_scale


        angles = zeros(4)

        for i=1:4
            angles[i] = asind((R_EARTH*sind(elev_angles[i]))/(R_EARTH + hi))
        end

        scale_f1 = (40.3/(f1)^2)*1e22
        scale_f2 = (40.3/(f2)^2)*1e22

        A1 = [cos(omega*sat_pose_t[1,4]) sin(omega*sat_pose_t[1,4]) 0;
              -sin(omega*sat_pose_t[1,4]) cos(omega*sat_pose_t[1,4]) 0;
              0 0 1];
        A2 = [cos(omega*sat_pose_t[2,4]) sin(omega*sat_pose_t[2,4]) 0;
              -sin(omega*sat_pose_t[2,4]) cos(omega*sat_pose_t[2,4]) 0;
              0 0 1];
        A3 = [cos(omega*sat_pose_t[3,4]) sin(omega*sat_pose_t[3,4]) 0;
              -sin(omega*sat_pose_t[3,4]) cos(omega*sat_pose_t[3,4]) 0;
              0 0 1];
        A4 = [cos(omega*sat_pose_t[4,4]) sin(omega*sat_pose_t[4,4]) 0;
              -sin(omega*sat_pose_t[4,4]) cos(omega*sat_pose_t[4,4]) 0;
              0 0 1];
        A5 = [cos(omega*sat_pose_t[5,4]) sin(omega*sat_pose_t[5,4]) 0;
              -sin(omega*sat_pose_t[5,4]) cos(omega*sat_pose_t[5,4]) 0;
              0 0 1];
        A6 = [cos(omega*sat_pose_t[6,4]) sin(omega*sat_pose_t[6,4]) 0;
              -sin(omega*sat_pose_t[6,4]) cos(omega*sat_pose_t[6,4]) 0;
              0 0 1];
        A7 = [cos(omega*sat_pose_t[7,4]) sin(omega*sat_pose_t[7,4]) 0;
              -sin(omega*sat_pose_t[7,4]) cos(omega*sat_pose_t[7,4]) 0;
              0 0 1];
        A8 = [cos(omega*sat_pose_t[8,4]) sin(omega*sat_pose_t[8,4]) 0;
              -sin(omega*sat_pose_t[8,4]) cos(omega*sat_pose_t[8,4]) 0;
              0 0 1];
        
        P = residual(initial_x0, sat_pose_t, elev_angles, f2)

        deltaP = -P

        #Create the A matrix to solve the LS problem
        
        A = zeros(8,8)

        A_row1 = LS_Amatrix(initial_x0, sat_pose_t[1,:], scale_f1, A1, angles[1])'
        A_row2 = LS_Amatrix(initial_x0, sat_pose_t[2,:], scale_f1, A2, angles[2])'
        A_row3 = LS_Amatrix(initial_x0, sat_pose_t[3,:], scale_f1, A3, angles[3])'
        A_row4 = LS_Amatrix(initial_x0, sat_pose_t[4,:], scale_f1, A4, angles[4])'
        A_row5 = LS_Amatrix(initial_x0, sat_pose_t[5,:], scale_f2, A5, angles[1])'
        A_row6 = LS_Amatrix(initial_x0, sat_pose_t[6,:], scale_f2, A6, angles[2])'
        A_row7 = LS_Amatrix(initial_x0, sat_pose_t[7,:], scale_f2, A7, angles[3])'
        A_row8 = LS_Amatrix(initial_x0, sat_pose_t[8,:], scale_f2, A8, angles[4])'

        
        A[1,:] = [A_row1[1:5]; zeros(3)]
        A[2,:] = [A_row2[1:4];0.0; A_row2[5]; zeros(2)]
        A[3,:] = [A_row3[1:4];zeros(2); A_row3[5]; 0.0]
        A[4,:] = [A_row4[1:4];zeros(3); A_row4[5]]
        A[5,:] = [A_row5[1:5]; zeros(3)]
        A[6,:] = [A_row6[1:4];0.0; A_row6[5]; zeros(2)]
        A[7,:] = [A_row7[1:4];zeros(2); A_row7[5]; 0.0]
        A[8,:] = [A_row8[1:4];zeros(3); A_row8[5]]
        
        F = qr(A)

        b = transpose(F.Q)*deltaP
        
        deltax = F.R\b
        
        initial_x0 = initial_x0+deltax


    end

    x_sol = initial_x0
    
    return x_sol
    
end

customjacobian (generic function with 1 method)

In [23]:
#pose1-4 are satellite locations and velocities
#r0 = [xyz of tag, freq offset]

#Find the true doppler measurments 

function Doppler_measurment(r0, pose1, pose2, pose3, pose4, time, z, zdot, f2)
    
    #Make sure to scale all positions
    
    omega = OMEGA_EARTH*time_scale_doppler #change to custom time scale
    
    omegahat = [0 -omega 0; omega 0 0; 0 0 0]

    #Ionosphere Errors at frequency 1
    Idot1 = (((40.3*r0[5])*((R_EARTH*sind(z[1]))*(R_EARTH*cosd(z[1])*zdot[1])/(R_EARTH + h)^2))/((f1^2)*(1-((R_EARTH*sind(z[1]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler
    Idot2 = (((40.3*r0[6])*((R_EARTH*sind(z[2]))*(R_EARTH*cosd(z[2])*zdot[2])/(R_EARTH + h)^2))/((f1^2)*(1-((R_EARTH*sind(z[2]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler
    Idot3 = (((40.3*r0[7])*((R_EARTH*sind(z[3]))*(R_EARTH*cosd(z[3])*zdot[3])/(R_EARTH + h)^2))/((f1^2)*(1-((R_EARTH*sind(z[3]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler
    Idot4 = (((40.3*r0[8])*((R_EARTH*sind(z[4]))*(R_EARTH*cosd(z[4])*zdot[4])/(R_EARTH + h)^2))/((f1^2)*(1-((R_EARTH*sind(z[4]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler

    #Ionosphere Errors at frequency 2
    Idot5 = (((40.3*r0[5])*((R_EARTH*sind(z[1]))*(R_EARTH*cosd(z[1])*zdot[1])/(R_EARTH + h)^2))/((f2^2)*(1-((R_EARTH*sind(z[1]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler
    Idot6 = (((40.3*r0[6])*((R_EARTH*sind(z[2]))*(R_EARTH*cosd(z[2])*zdot[2])/(R_EARTH + h)^2))/((f2^2)*(1-((R_EARTH*sind(z[2]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler
    Idot7 = (((40.3*r0[7])*((R_EARTH*sind(z[3]))*(R_EARTH*cosd(z[3])*zdot[3])/(R_EARTH + h)^2))/((f2^2)*(1-((R_EARTH*sind(z[3]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler
    Idot8 = (((40.3*r0[8])*((R_EARTH*sind(z[4]))*(R_EARTH*cosd(z[4])*zdot[4])/(R_EARTH + h)^2))/((f2^2)*(1-((R_EARTH*sind(z[4]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler

    #Rotation matrices for rotation of Earth effect
    
    A1 = [cos(omega*time[1]) sin(omega*time[1]) 0;
          -sin(omega*time[1]) cos(omega*time[1]) 0;
          0 0 1];
    A2 = [cos(omega*time[2]) sin(omega*time[2]) 0;
          -sin(omega*time[2]) cos(omega*time[2]) 0;
          0 0 1];
    A3 = [cos(omega*time[3]) sin(omega*time[3]) 0;
          -sin(omega*time[3]) cos(omega*time[3]) 0;
          0 0 1];
    A4 = [cos(omega*time[4]) sin(omega*time[4]) 0;
          -sin(omega*time[4]) cos(omega*time[4]) 0;
          0 0 1];
    
    #r0[4] is scaled coming in 
    
    #Measurments at frequency 1
    deltaf1 = (f1/c_doppler)*(0.5*(1/norm(r0[1:3] - A1*pose1[1:3]))*(-2*r0[1:3]'*A1*omegahat*pose1[1:3] - 2*r0[1:3]'*A1*pose1[4:6]+pose1[1:3]'*pose1[4:6]+pose1[4:6]'*pose1[1:3]) + Idot1 + r0[4])
    deltaf2 = (f1/c_doppler)*(0.5*(1/norm(r0[1:3] - A2*pose2[1:3]))*(-2*r0[1:3]'*A2*omegahat*pose2[1:3] - 2*r0[1:3]'*A2*pose2[4:6]+pose2[1:3]'*pose2[4:6]+pose2[4:6]'*pose2[1:3]) + Idot2 + r0[4])
    deltaf3 = (f1/c_doppler)*(0.5*(1/norm(r0[1:3] - A3*pose3[1:3]))*(-2*r0[1:3]'*A3*omegahat*pose3[1:3] - 2*r0[1:3]'*A3*pose3[4:6]+pose3[1:3]'*pose3[4:6]+pose3[4:6]'*pose3[1:3]) + Idot3 + r0[4])
    deltaf4 = (f1/c_doppler)*(0.5*(1/norm(r0[1:3] - A4*pose4[1:3]))*(-2*r0[1:3]'*A4*omegahat*pose4[1:3] - 2*r0[1:3]'*A4*pose4[4:6]+pose4[1:3]'*pose4[4:6]+pose4[4:6]'*pose4[1:3]) + Idot4 + r0[4])
    
    #Measurments at frequency 2
    deltaf5 = (f2/c_doppler)*(0.5*(1/norm(r0[1:3] - A1*pose1[1:3]))*(-2*r0[1:3]'*A1*omegahat*pose1[1:3] - 2*r0[1:3]'*A1*pose1[4:6]+pose1[1:3]'*pose1[4:6]+pose1[4:6]'*pose1[1:3]) + Idot5 + r0[4])
    deltaf6 = (f2/c_doppler)*(0.5*(1/norm(r0[1:3] - A2*pose2[1:3]))*(-2*r0[1:3]'*A2*omegahat*pose2[1:3] - 2*r0[1:3]'*A2*pose2[4:6]+pose2[1:3]'*pose2[4:6]+pose2[4:6]'*pose2[1:3]) + Idot6 + r0[4])
    deltaf7 = (f2/c_doppler)*(0.5*(1/norm(r0[1:3] - A3*pose3[1:3]))*(-2*r0[1:3]'*A3*omegahat*pose3[1:3] - 2*r0[1:3]'*A3*pose3[4:6]+pose3[1:3]'*pose3[4:6]+pose3[4:6]'*pose3[1:3]) + Idot7 + r0[4])
    deltaf8 = (f2/c_doppler)*(0.5*(1/norm(r0[1:3] - A4*pose4[1:3]))*(-2*r0[1:3]'*A4*omegahat*pose4[1:3] - 2*r0[1:3]'*A4*pose4[4:6]+pose4[1:3]'*pose4[4:6]+pose4[4:6]'*pose4[1:3]) + Idot8 + r0[4])

    return [deltaf1, deltaf2, deltaf3, deltaf4, deltaf5, deltaf6, deltaf7, deltaf8]
    
end

Doppler_measurment (generic function with 1 method)

In [24]:
#Used to create the A matrix to propogate the covariance of the satellites to the covariance of the tag

function createA_TOA(sat_pose_t, elev_angles, x_sol, f2)
    
    omega = OMEGA_EARTH*time_scale #scaled

    angles = zeros(4)

    for i=1:4
        angles[i] = asind((R_EARTH*sind(elev_angles[i]))/(R_EARTH + hi))
    end

    scale_f1 = (40.3/(f1)^2)*1e22
    scale_f2 = (40.3/(f2)^2)*1e22

    A1 = [cos(omega*sat_pose_t[1,4]) sin(omega*sat_pose_t[1,4]) 0;
          -sin(omega*sat_pose_t[1,4]) cos(omega*sat_pose_t[1,4]) 0;
          0 0 1];
    A2 = [cos(omega*sat_pose_t[2,4]) sin(omega*sat_pose_t[2,4]) 0;
          -sin(omega*sat_pose_t[2,4]) cos(omega*sat_pose_t[2,4]) 0;
          0 0 1];
    A3 = [cos(omega*sat_pose_t[3,4]) sin(omega*sat_pose_t[3,4]) 0;
          -sin(omega*sat_pose_t[3,4]) cos(omega*sat_pose_t[3,4]) 0;
          0 0 1];
    A4 = [cos(omega*sat_pose_t[4,4]) sin(omega*sat_pose_t[4,4]) 0;
          -sin(omega*sat_pose_t[4,4]) cos(omega*sat_pose_t[4,4]) 0;
          0 0 1];
    A5 = [cos(omega*sat_pose_t[5,4]) sin(omega*sat_pose_t[5,4]) 0;
          -sin(omega*sat_pose_t[5,4]) cos(omega*sat_pose_t[5,4]) 0;
          0 0 1];
    A6 = [cos(omega*sat_pose_t[6,4]) sin(omega*sat_pose_t[6,4]) 0;
          -sin(omega*sat_pose_t[6,4]) cos(omega*sat_pose_t[6,4]) 0;
          0 0 1];
    A7 = [cos(omega*sat_pose_t[7,4]) sin(omega*sat_pose_t[7,4]) 0;
          -sin(omega*sat_pose_t[7,4]) cos(omega*sat_pose_t[7,4]) 0;
          0 0 1];
    A8 = [cos(omega*sat_pose_t[8,4]) sin(omega*sat_pose_t[8,4]) 0;
          -sin(omega*sat_pose_t[8,4]) cos(omega*sat_pose_t[8,4]) 0;
          0 0 1];

    #create dfdy
    dfdx = zeros(8,8)

    dfdx_row1 = LS_Amatrix(x_sol, sat_pose_t[1,:], scale_f1, A1, angles[1])'
    dfdx_row2 = LS_Amatrix(x_sol, sat_pose_t[2,:], scale_f1, A2, angles[2])'
    dfdx_row3 = LS_Amatrix(x_sol, sat_pose_t[3,:], scale_f1, A3, angles[3])'
    dfdx_row4 = LS_Amatrix(x_sol, sat_pose_t[4,:], scale_f1, A4, angles[4])'
    dfdx_row5 = LS_Amatrix(x_sol, sat_pose_t[5,:], scale_f2, A5, angles[1])'
    dfdx_row6 = LS_Amatrix(x_sol, sat_pose_t[6,:], scale_f2, A6, angles[2])'
    dfdx_row7 = LS_Amatrix(x_sol, sat_pose_t[7,:], scale_f2, A7, angles[3])'
    dfdx_row8 = LS_Amatrix(x_sol, sat_pose_t[8,:], scale_f2, A8, angles[4])'

    dfdx[1,:] = [dfdx_row1[1:5]; zeros(3)]
    dfdx[2,:] = [dfdx_row2[1:4];0.0; dfdx_row2[5]; zeros(2)]
    dfdx[3,:] = [dfdx_row3[1:4];zeros(2); dfdx_row3[5]; 0.0]
    dfdx[4,:] = [dfdx_row4[1:4];zeros(3); dfdx_row4[5]]
    dfdx[5,:] = [dfdx_row5[1:5]; zeros(3)]
    dfdx[6,:] = [dfdx_row6[1:4];0.0; dfdx_row6[5]; zeros(2)]
    dfdx[7,:] = [dfdx_row7[1:4];zeros(2); dfdx_row7[5]; 0.0]
    dfdx[8,:] = [dfdx_row8[1:4];zeros(3); dfdx_row8[5]]

    #derivative wrt measurements (satellite position and times)
    dfdy = zeros(8,32)

    #create dfdx
    dfdy[1, 1:4] = LS_Amatrix_dpdy(x_sol, sat_pose_t[1,:], scale_f1, A1, angles[1])'[1:4]
    dfdy[2, 5:8] = LS_Amatrix_dpdy(x_sol, sat_pose_t[2,:], scale_f1, A2, angles[2])'[1:4]
    dfdy[3, 9:12] = LS_Amatrix_dpdy(x_sol, sat_pose_t[3,:], scale_f1, A3, angles[3])'[1:4]
    dfdy[4, 13:16] = LS_Amatrix_dpdy(x_sol, sat_pose_t[4,:], scale_f1, A4, angles[4])'[1:4]
    dfdy[5, 17:20] = LS_Amatrix_dpdy(x_sol, sat_pose_t[5,:], scale_f2, A5, angles[1])'[1:4]
    dfdy[6, 21:24] = LS_Amatrix_dpdy(x_sol, sat_pose_t[6,:], scale_f2, A6, angles[2])'[1:4]
    dfdy[7, 25:28] = LS_Amatrix_dpdy(x_sol, sat_pose_t[7,:], scale_f2, A7, angles[3])'[1:4]
    dfdy[8, 29:32] = LS_Amatrix_dpdy(x_sol, sat_pose_t[8,:], scale_f2, A8, angles[4])'[1:4]

    A = -inv(dfdx)*dfdy
    
    return A
end

createA_TOA (generic function with 1 method)

In [25]:
#scaled

#This is a random distrubution to get the TECV daytime values 
using Distributions
mu = 8e-6
sigma = 3e-6
lb = 3e-6
ub = 13e-6
d = Truncated(Normal(mu,sigma), lb, ub)

hi = 350e3 #in meters

350000.0

In [26]:
#calculate the covariance of the tag from the TOA method 

function calculate_covariance_TOA(sat_poses, elev_angles, guess, P_TOA, r0_scaled, f2)
    TEC = rand(d,4) 
    r0_TEC= vec([r0_scaled TEC])
    sat_poses_scaled = sat_poses/distance_scale #scale the data
    t = get_time(r0_TEC, sat_poses_scaled, elev_angles, f2)
    sat_poses_2 = vcat(sat_poses_scaled, sat_poses_scaled)
    sat_pose_t = [sat_poses_2 t]
    
    #initial guess
    
    initial_x0 = [guess[1]/distance_scale, guess[2]/distance_scale, guess[3]/distance_scale, 0.0001/time_scale, 3e-6, 3.5e-6, 2e-6, 4e-6]

    #get the converged solution
    x_sol = customjacobian(initial_x0,sat_pose_t, elev_angles, f2)
    
    if all(isfinite, x_sol) == true
    
        A = createA_TOA(sat_pose_t, elev_angles, x_sol, f2)

        #convert the covariance of the satellites to the covariance of the tag

        covariance_tag = A*P_TOA*transpose(A)

        #Only get the pose covariance
        pose_covariance = covariance_tag[1:3, 1:3]

        #scale the pose covariance to meters
        pose_covariance_scaled = covariance_tag[1:3, 1:3]*(1e3*distance_scale)^2

        #longitude and latitude of the tag position. 
        lambda = -165.4545 #longitude 
        phi =  0 #latitude

        #Switch from xyz to East North Up coordinate system

        R = [-sind(lambda) cosd(lambda) 0; 
            -sind(phi)*cosd(lambda) -sind(phi)*sind(lambda) cosd(phi);
            cosd(phi)*cosd(lambda) cosd(phi)*sind(lambda) sind(phi)]

        #Rotate the covariance matrix
        covariance_matrix_scaled_rotated = R*pose_covariance_scaled*R'

        #Find the standard deviation of the tag position
        PDOP_LS = sqrt(covariance_tag[1,1] + covariance_tag[2,2] + covariance_tag[3,3])

        #This is the standard deviation of the tag postion scaled in meters and seconds
        PDOP_LS_scaled = PDOP_LS*distance_scale*1e3
 
    else
        
        PDOP_LS = 0
        PDOP_LS_scaled = eyes(8)
        
    end
    
    return PDOP_LS_scaled, covariance_matrix_scaled_rotated
    
end

calculate_covariance_TOA (generic function with 1 method)

In [27]:
#sat_pose_f is a vector of all the satellite positions, velocities

function doppler_residual_vec(x, sat_pose_f, time, z, zdot, f2)
    
    omega = OMEGA_EARTH*time_scale_doppler #transform to custom scale
    
    omegahat = [0 -omega 0; omega 0 0; 0 0 0]
    
    A1 = [cos(omega*time[1]) sin(omega*time[1]) 0;
          -sin(omega*time[1]) cos(omega*time[1]) 0;
          0 0 1];
    A2 = [cos(omega*time[2]) sin(omega*time[2]) 0;
          -sin(omega*time[2]) cos(omega*time[2]) 0;
          0 0 1];
    A3 = [cos(omega*time[3]) sin(omega*time[3]) 0;
          -sin(omega*time[3]) cos(omega*time[3]) 0;
          0 0 1];
    A4 = [cos(omega*time[4]) sin(omega*time[4]) 0;
          -sin(omega*time[4]) cos(omega*time[4]) 0;
          0 0 1];
    
    #bdot is a velocity. therefore it is scaled
    #Ionosphere Errors at frequency 1
    Idot1 = (((40.3*x[5])*((R_EARTH*sind(z[1]))*(R_EARTH*cosd(z[1])*zdot[1])/(R_EARTH + h)^2))/((f1^2)*(1-((R_EARTH*sind(z[1]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler
    Idot2 = (((40.3*x[6])*((R_EARTH*sind(z[2]))*(R_EARTH*cosd(z[2])*zdot[2])/(R_EARTH + h)^2))/((f1^2)*(1-((R_EARTH*sind(z[2]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler
    Idot3 = (((40.3*x[7])*((R_EARTH*sind(z[3]))*(R_EARTH*cosd(z[3])*zdot[3])/(R_EARTH + h)^2))/((f1^2)*(1-((R_EARTH*sind(z[3]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler
    Idot4 = (((40.3*x[8])*((R_EARTH*sind(z[4]))*(R_EARTH*cosd(z[4])*zdot[4])/(R_EARTH + h)^2))/((f1^2)*(1-((R_EARTH*sind(z[4]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler

    #Ionosphere Errors at frequency 2
    Idot5 = (((40.3*x[5])*((R_EARTH*sind(z[1]))*(R_EARTH*cosd(z[1])*zdot[1])/(R_EARTH + h)^2))/((f2^2)*(1-((R_EARTH*sind(z[1]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler
    Idot6 = (((40.3*x[6])*((R_EARTH*sind(z[2]))*(R_EARTH*cosd(z[2])*zdot[2])/(R_EARTH + h)^2))/((f2^2)*(1-((R_EARTH*sind(z[2]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler
    Idot7 = (((40.3*x[7])*((R_EARTH*sind(z[3]))*(R_EARTH*cosd(z[3])*zdot[3])/(R_EARTH + h)^2))/((f2^2)*(1-((R_EARTH*sind(z[3]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler
    Idot8 = (((40.3*x[8])*((R_EARTH*sind(z[4]))*(R_EARTH*cosd(z[4])*zdot[4])/(R_EARTH + h)^2))/((f2^2)*(1-((R_EARTH*sind(z[4]))/(R_EARTH+h))^2)^1.5))*1e20*1e-3/velocity_scale_doppler

    #Measurments at frequency 1
    res1 = (f1/c_doppler)*(0.5*(1/norm(x[1:3] - A1*sat_pose_f[1:3]))*(-2*x[1:3]'*A1*omegahat*sat_pose_f[1:3] - 2*x[1:3]'*A1*sat_pose_f[4:6]+sat_pose_f[1:3]'*sat_pose_f[4:6]+sat_pose_f[4:6]'*sat_pose_f[1:3]) + Idot1 + x[4]) - sat_pose_f[7]
    res2 = (f1/c_doppler)*(0.5*(1/norm(x[1:3] - A2*sat_pose_f[8:10]))*(-2*x[1:3]'*A2*omegahat*sat_pose_f[8:10] - 2*x[1:3]'*A2*sat_pose_f[11:13]+sat_pose_f[8:10]'*sat_pose_f[11:13]+sat_pose_f[11:13]'*sat_pose_f[8:10]) + Idot2 + x[4]) - sat_pose_f[14]
    res3 = (f1/c_doppler)*(0.5*(1/norm(x[1:3] - A3*sat_pose_f[15:17]))*(-2*x[1:3]'*A3*omegahat*sat_pose_f[15:17] - 2*x[1:3]'*A3*sat_pose_f[18:20]+sat_pose_f[15:17]'*sat_pose_f[18:20]+sat_pose_f[18:20]'*sat_pose_f[15:17]) + Idot3 + x[4]) - sat_pose_f[21]
    res4 = (f1/c_doppler)*(0.5*(1/norm(x[1:3] - A4*sat_pose_f[22:24]))*(-2*x[1:3]'*A4*omegahat*sat_pose_f[22:24] - 2*x[1:3]'*A4*sat_pose_f[25:27]+sat_pose_f[22:24]'*sat_pose_f[25:27]+sat_pose_f[25:27]'*sat_pose_f[22:24]) + Idot4 + x[4]) - sat_pose_f[28]
    
    #Measurments at frequency 2
    res5 = (f2/c_doppler)*(0.5*(1/norm(x[1:3] - A1*sat_pose_f[1:3]))*(-2*x[1:3]'*A1*omegahat*sat_pose_f[1:3] - 2*x[1:3]'*A1*sat_pose_f[4:6]+sat_pose_f[1:3]'*sat_pose_f[4:6]+sat_pose_f[4:6]'*sat_pose_f[1:3]) + Idot5 + x[4]) - sat_pose_f[35]
    res6 = (f2/c_doppler)*(0.5*(1/norm(x[1:3] - A2*sat_pose_f[8:10]))*(-2*x[1:3]'*A2*omegahat*sat_pose_f[8:10] - 2*x[1:3]'*A2*sat_pose_f[11:13]+sat_pose_f[8:10]'*sat_pose_f[11:13]+sat_pose_f[11:13]'*sat_pose_f[8:10]) + Idot6 + x[4]) - sat_pose_f[42]
    res7 = (f2/c_doppler)*(0.5*(1/norm(x[1:3] - A3*sat_pose_f[15:17]))*(-2*x[1:3]'*A3*omegahat*sat_pose_f[15:17] - 2*x[1:3]'*A3*sat_pose_f[18:20]+sat_pose_f[15:17]'*sat_pose_f[18:20]+sat_pose_f[18:20]'*sat_pose_f[15:17]) + Idot7 + x[4]) - sat_pose_f[49]
    res8 = (f2/c_doppler)*(0.5*(1/norm(x[1:3] - A4*sat_pose_f[22:24]))*(-2*x[1:3]'*A4*omegahat*sat_pose_f[22:24] - 2*x[1:3]'*A4*sat_pose_f[25:27]+sat_pose_f[22:24]'*sat_pose_f[25:27]+sat_pose_f[25:27]'*sat_pose_f[22:24]) + Idot8 + x[4]) - sat_pose_f[56]

    
    return [res1; res2; res3; res4; res5; res6; res7; res8]
   
end

doppler_residual_vec (generic function with 1 method)

In [28]:
#Create the A matrix for Doppler to obtain the covariance of the tag

function createA_Doppler(sat_pose_t, zenith_angles, x_sol, time, zdot, f2)
    satpose_vec = vec(sat_pose_t')
    dfdy_fd = ForwardDiff.jacobian(dall_sats_scaled -> doppler_residual_vec(x_sol, dall_sats_scaled, time, zenith_angles, zdot, f2), satpose_vec)
    dfdx_fd = ForwardDiff.jacobian(dx -> doppler_residual_vec(dx, satpose_vec, time, zenith_angles, zdot, f2), x_sol)
    
    #use implicit function theorem
    A = -inv(dfdx_fd)*dfdy_fd
    
    return A
end

createA_Doppler (generic function with 1 method)

In [29]:
#Solve the Doppler LS problem
function customjacobian_Doppler(initial_x0, sat_pose_t, time, zenith_angles, zdot, f2)

    #make sure sat_pose_t is a vector
    satpose_vec = vec(sat_pose_t')
    
    while norm(doppler_residual_vec(initial_x0, satpose_vec, time, zenith_angles, zdot, f2))>1e-10
        
        res = doppler_residual_vec(initial_x0, satpose_vec, time, zenith_angles, zdot, f2)
    
        deltaP = -res
        
        A = ForwardDiff.jacobian(dx -> doppler_residual_vec(dx, satpose_vec, time, zenith_angles, zdot, f2), initial_x0)
        
        F = qr(A)

        d = transpose(F.Q)*deltaP

        deltax = F.R\d
        
        initial_x0 = initial_x0+deltax
        
    end
    
    x_sol = initial_x0
    
    return x_sol
        
end

customjacobian_Doppler (generic function with 1 method)

In [30]:
function calculate_covariance_Doppler(pose1, pose2, pose3, pose4, zenith_angles, zdot, P_Doppler, r0_scaled_Doppler, f2)
    
    #the difference added to actual solution to converge (for Doppler scenerio)
    diff = 50e3*1e-3/distance_scale_doppler 
    
    #assume a propogation time to include rotation of earth
    time = [0.006, 0.006, 0.006, 0.008]/time_scale_doppler
    
    #initial guess for Doppler algorithm
    initial_x0 = [r0_scaled_Doppler[1]+diff, r0_scaled_Doppler[2]+diff, r0_scaled_Doppler[3]+diff, 1.002*1e4*1e-3/velocity_scale_doppler, 3.1e-4, 4.2e-4, 3.2e-4, 4.7e-4]
    
    #Obtain the true measurments for the doppler case
    deltaf = Doppler_measurment(r0_scaled_Doppler, pose1, pose2, pose3, pose4, time, zenith_angles, zdot, f2)
    
    #augment the frequency measurments to the satellite position and velocities
    pose1_rdot1 = [pose1 deltaf[1]]
    pose2_rdot2 = [pose2 deltaf[2]]
    pose3_rdot3 = [pose3 deltaf[3]]
    pose4_rdot4 = [pose4 deltaf[4]]
    
    pose5_rdot5= [pose1 deltaf[5]]
    pose6_rdot6= [pose2 deltaf[6]]
    pose7_rdot7= [pose3 deltaf[7]]
    pose8_rdot8= [pose4 deltaf[8]]
    
    all_sats_scaled = vcat(pose1_rdot1,pose2_rdot2,pose3_rdot3,pose4_rdot4, pose5_rdot5,pose6_rdot6,pose7_rdot7,pose8_rdot8)
    
    #Find the Least Squares solution
    x_sol = customjacobian_Doppler(initial_x0, all_sats_scaled, time, zenith_angles, zdot, f2)
    
    if all(isfinite, x_sol) == true
        
    
        #Get the A matrix to find the covaraince of the tag
        A = createA_Doppler(all_sats_scaled, zenith_angles, x_sol, time, zdot, f2)

        #Calculate the Covariance of the tag
        covariance_tag = A*P_Doppler*transpose(A)

        #Only get the part of the covariance matrix that has to do with position
        pose_covariance = covariance_tag[1:3, 1:3]

        #Scale back to meters and seconds
        pose_covariance_scaled = covariance_tag[1:3, 1:3]*(1e3*distance_scale_doppler)^2

        lambda = -165.4545 #longitude 
        phi =  0 #latitude

        #Rotation matrix to update the covariance from xyz to east north up coordinate system
        R = [-sind(lambda) cosd(lambda) 0; 
            -sind(phi)*cosd(lambda) -sind(phi)*sind(lambda) cosd(phi);
            cosd(phi)*cosd(lambda) cosd(phi)*sind(lambda) sind(phi)]

        covariance_matrix_scaled_rotated = R*pose_covariance_scaled*R'

        PDOP_LS = sqrt(covariance_tag[1,1] + covariance_tag[2,2] + covariance_tag[3,3])

        PDOP_LS_scaled = PDOP_LS*distance_scale_doppler*1e3

        #return PDOP_LS_scaled, covariance_matrix_scaled_rotated
        
    else
        
        PDOP_LS_scaled = 0
        covariance_matrix_scaled_rotated = eyes(8)
    end
        
        
    return PDOP_LS_scaled, covariance_matrix_scaled_rotated
    
end

calculate_covariance_Doppler (generic function with 1 method)

In [31]:
#DRAW THE ELLIPSE
function draw_ellipse(covariance_matrix)
    
    eig_value = eigvals(covariance_matrix)
    eig_vecs = eigvecs(covariance_matrix)
    
    max_eigval_index = argmax(eig_value)
    min_eigval_index = argmin(eig_value)

    max_eigval = eig_value[max_eigval_index]
    min_eigval = eig_value[min_eigval_index]
    
    max_eigvec = eig_vecs[:, max_eigval_index]
    min_eigvec = eig_vecs[:, min_eigval_index]
    
    angle = atan(max_eigvec[2], max_eigvec[1])
    
    if angle<0
        angle = angle + 2*pi;
    end
    
    #95% confidence interval
    chisquare_val = 2.4477
    theta_grid = 0:0.01:2*pi

    phi = angle;

    a = chisquare_val*sqrt(max_eigval)
    b = chisquare_val*sqrt(min_eigval)

    ellipse_x = zeros(size(theta_grid)[1], 1)
    ellipse_y = zeros(size(theta_grid)[1], 1)

    for i=1:size(theta_grid)[1]

        #Get ellipse coordinates 
        ellipse_x[i,1] = a*cos(theta_grid[i])
        ellipse_y[i,1] = b*sin(theta_grid[i])

    end
    
    R = [cos(phi) sin(phi); -sin(phi) cos(phi)];
    r_ellipse = [ellipse_x ellipse_y]*R
    
    return r_ellipse
    
end

draw_ellipse (generic function with 1 method)

In [32]:
#LOAD the Figure

#Initialize tag latitude
initial_tag_latitude = 0
r0, r0_scaled_TOA, r0_scaled_Doppler = tag_xyz(initial_tag_latitude)

#Load the image of the Earth
earth = load(download("https://svs.gsfc.nasa.gov/vis/a000000/a002900/a002915/bluemarble-2048.png"));

#variables to save the final covariance
pose_covariance_result_TOA= zeros(3,3)

pose_covariance_result_Doppler= zeros(3,3)

#Plot the Earth
#this was command before, now its saying unknown 3f0?
#fig, ax, earth = mesh(Sphere(Point3f0(0), 6371f0), color= earth, figure = (resolution = (1200,1200),))
fig, ax, earth = mesh(Sphere(Point3f(0), 6371f0), color= earth, figure = (resolution = (1200,1200),))


#Chooses the part of the figure to put all the sliders
parameters_fig = fig[1,2]

#Chooses the part of the figure to put the covariance ellipses
ellipse_fig = parameters_fig[2,1]


#Define all the Observables (values that change once the sliders are changed)

freq_offset = Observable(200)

#Print the PDOP
PDOP_TOA_text = Observable("test")
PDOP_Doppler_text = Observable("test")

#tag latitude
tag_lat = Observable(0)
#tag xyz in km
#this was command before, now its saying unknown 3f0?
#tag_xyzt = Observable(Point3f0(r0[1:3]))
tag_xyzt = Observable(Point3f(r0[1:3]))


#Where along the orbit is the constellation
along = Observable(Int(1))

#Ellipse x and y for East North direction for TOA case
ellipse_xy1_TOA = Observable(zeros(629))
ellipse_xy2_TOA = Observable(zeros(629))

#Ellipse x and y for East North direction for TOA case
ellipse_xz1_TOA = Observable(zeros(629))
ellipse_xz2_TOA = Observable(zeros(629))

#Ellipse x and y for East North direction for TOA case
ellipse_yz1_TOA = Observable(zeros(629))
ellipse_yz2_TOA = Observable(zeros(629))

#Ellipse x and y for East North direction for Doppler case
ellipse_xy1_Doppler = Observable(zeros(629))
ellipse_xy2_Doppler = Observable(zeros(629))

#Ellipse x and y for East North direction for Doppler case
ellipse_xz1_Doppler = Observable(zeros(629))
ellipse_xz2_Doppler = Observable(zeros(629))

#Ellipse x and y for East North direction for Doppler case
ellipse_yz1_Doppler = Observable(zeros(629))
ellipse_yz2_Doppler = Observable(zeros(629))

#orbit x values
eci_1_x = Observable(eci_1_s[1,:])
eci_2_x = Observable(eci_2_s[1,:])
eci_3_x = Observable(eci_3_s[1,:])
eci_4_x = Observable(eci_4_s[1,:])

#orbit y values
eci_1_y = Observable(eci_1_s[2,:])
eci_2_y = Observable(eci_2_s[2,:])
eci_3_y = Observable(eci_3_s[2,:])
eci_4_y = Observable(eci_4_s[2,:])

#orbit z values
eci_1_z = Observable(eci_1_s[3,:])
eci_2_z = Observable(eci_2_s[3,:])
eci_3_z = Observable(eci_3_s[3,:])
eci_4_z = Observable(eci_4_s[3,:])

#satellite positions
#this was command before, now its saying unknown 3f0?
# sats = Observable([Point3f0(eci_1_s[1,1],eci_1_s[2,1],eci_1_s[3,1]), 
#                    Point3f0(eci_2_s[1,1],eci_2_s[2,1],eci_2_s[3,1]),
#                    Point3f0(eci_3_s[1,1],eci_3_s[2,1],eci_3_s[3,1]),
#                    Point3f0(eci_4_s[1,1],eci_4_s[2,1],eci_4_s[3,1])])
sats = Observable([Point3f(eci_1_s[1,1],eci_1_s[2,1],eci_1_s[3,1]), 
                   Point3f(eci_2_s[1,1],eci_2_s[2,1],eci_2_s[3,1]),
                   Point3f(eci_3_s[1,1],eci_3_s[2,1],eci_3_s[3,1]),
                   Point3f(eci_4_s[1,1],eci_4_s[2,1],eci_4_s[3,1])])


#Initial guess
#this was command before, now its saying unknown 3f0?
#guess = Observable(Point3f0(eci_1_s[1,1],eci_1_s[2,1],eci_1_s[3,1]))
guess = Observable(Point3f(eci_1_s[1,1],eci_1_s[2,1],eci_1_s[3,1]))


#Timing accuracy and frequency accuracy
timing_accuracy= Observable(10.0)
frequency_accuracy = Observable(1.0)

#time_range = [1e-12, 1e-11, 1e-10]
#freq_range = [1e-6, 1e-5, 1e-4] 

#Add in the sliders
lsgrid = labelslidergrid!(
    fig,
    ["Orbit Altitude", "RAAN Seperation", "True Anomaly Seperation", "Delta True Anomaly Seperation", "Time Step ", "Time Accuracy", "Frequency Accuracy", "Tag Latitude", "Frequency Offset"],
    #[400:100:1200, 1:1:5, 1:0.5:15, 1:1:5, 1:10:1000, -12:1:-6, -6:1:1, 0:10:70, 50:50:500];
    [400:100:1200, 1:1:5, 1:0.5:15, 1:1:5, 1:10:1000, 10:10:100, 100:100:1000, 0:10:70, 50:50:500];
    formats = [x -> "$(round(x, digits = 1))$s" for s in ["km", "Degrees", "Degrees", "Degrees", " ", "ns", "mHz", "Degrees", "Hz"]],
    width = 500,
    tellheight = false)

#Placement for the sliders
parameters_fig[1,1] = lsgrid.layout


#Code to include buttons
# button_fig = ellipse_fig[3,1]
# #creating a button (works)
# TOA_btn = Button(button_fig[1,1]; label = "TOA 2 freq", tellwidth = false)
# Doppler_btn = Button(button_fig[1,3]; label = "Doppler 2 freq", tellwidth = false)

# isTOA = Observable(false)
# isDoppler = Observable(false)

# on(TOA_btn.clicks) do clicks; 
#     isTOA[] = true 
#     #update the scale
#     isDoppler[] = false; 
# end

# on(Doppler_btn.clicks) do clicks; 
#     isDoppler[] = true 
#     #update the scale

#     isTOA[] = false; 
# end

# ellipse_fig[3,1] = buttongrid = GridLayout(tellwidth= false)

# labels = ["ToA 2 frequencies, Doppler 2 frequencies"]

# buttons = buttongrid[1, 1:2] = [
#     Button(fig, 
#         label = l, height=60, width = 250, textsize = 30
    
#     )
#     for l in labels
# ]

#Records the change in each slider and updates the values
sliderobservables = [s.value for s in lsgrid.sliders]
bars = lift(sliderobservables...) do slvalues...
    values = [slvalues...]
    altitude = values[1]
    RAAN_sep = values[2]
    true_anom = values[3]
    delta_true_anom = values[4]
    alongorbit = values[5]
    time_acc = values[6]
    freq_acc = values[7]
    tag_latitude = values[8]
    f2_observable = values[9]
    
    #update the orbit based off of the updated parameters
    eci_1_update, eci_2_update, eci_3_update, eci_4_update, xyz_ = orbit_update(true_anom, RAAN_sep, delta_true_anom, altitude);
    
    #println("this is tag_latitude: ", tag_latitude)
    r0_tag_update, r0_scaled_TOA_update, r0_scaled_Doppler_update = tag_xyz(tag_latitude)
    
    #update the observables
    
    tag_xyzt[] = r0_tag_update[1:3]
    
    eci_1_x[] = eci_1_update[1,:]
    eci_2_x[] = eci_2_update[1,:]
    eci_3_x[] = eci_3_update[1,:]
    eci_4_x[] = eci_4_update[1,:]
    
    eci_1_y[] = eci_1_update[2,:]
    eci_2_y[] = eci_2_update[2,:]
    eci_3_y[] = eci_3_update[2,:]
    eci_4_y[] = eci_4_update[2,:]
    
    eci_1_z[] = eci_1_update[3,:]
    eci_2_z[] = eci_2_update[3,:]
    eci_3_z[] = eci_3_update[3,:]
    eci_4_z[] = eci_4_update[3,:]
    
    
    timing_accuracy[] = time_acc
    frequency_accuracy[] = freq_acc
    tag_lat[] = tag_latitude
    
    freq_offset[] = f2_observable
    
    along[] = alongorbit 
    
    intorbit = Int(alongorbit)
    
    #sat_poses is in km
    sat_poses = [eci_1_update[1,intorbit] eci_1_update[2,intorbit] eci_1_update[3,intorbit]; 
                 eci_2_update[1,intorbit] eci_2_update[2,intorbit] eci_2_update[3,intorbit];
                 eci_3_update[1,intorbit] eci_3_update[2,intorbit] eci_3_update[3,intorbit];
                 eci_4_update[1,intorbit] eci_4_update[2,intorbit] eci_4_update[3,intorbit]]
    
    sat_poses2 = [eci_1_update[1,intorbit+1] eci_1_update[2,intorbit+1] eci_1_update[3,intorbit+1]; 
                 eci_2_update[1,intorbit+1] eci_2_update[2,intorbit+1] eci_2_update[3,intorbit+1];
                 eci_3_update[1,intorbit+1] eci_3_update[2,intorbit+1] eci_3_update[3,intorbit+1];
                 eci_4_update[1,intorbit+1] eci_4_update[2,intorbit+1] eci_4_update[3,intorbit+1]]
    
    sat_velocity = [eci_1_update[4,intorbit] eci_1_update[5,intorbit] eci_1_update[6,intorbit]; 
                 eci_2_update[4,intorbit] eci_2_update[5,intorbit] eci_2_update[6,intorbit];
                 eci_3_update[4,intorbit] eci_3_update[5,intorbit] eci_3_update[6,intorbit];
                 eci_4_update[4,intorbit] eci_4_update[5,intorbit] eci_4_update[6,intorbit]]
    
    #scale the sat poses and velocities
    pose1 = [sat_poses[1,:]'/distance_scale_doppler sat_velocity[1,:]'/velocity_scale_doppler]
    pose2 = [sat_poses[2,:]'/distance_scale_doppler sat_velocity[2,:]'/velocity_scale_doppler]
    pose3 = [sat_poses[3,:]'/distance_scale_doppler sat_velocity[3,:]'/velocity_scale_doppler]
    pose4 = [sat_poses[4,:]'/distance_scale_doppler sat_velocity[4,:]'/velocity_scale_doppler]
    
    zenith_angles = elevation_angle(sat_poses*1e3, r0[1:3]*1e3)
    zenith_angles2 = elevation_angle(sat_poses2*1e3, r0[1:3]*1e3)
    
    zdot = zenith_angles - zenith_angles2
    
    #update the sat observable
    #3f0 now causing error
#     sats[] = [Point3f0(eci_1_update[1,intorbit],eci_1_update[2,intorbit],eci_1_update[3,intorbit]), 
#                    Point3f0(eci_2_update[1,intorbit],eci_2_update[2,intorbit],eci_2_update[3,intorbit]),
#                    Point3f0(eci_3_update[1,intorbit],eci_3_update[2,intorbit],eci_3_update[3,intorbit]),
#                    Point3f0(eci_4_update[1,intorbit],eci_4_update[2,intorbit],eci_4_update[3,intorbit])]
    
    sats[] = [Point3f(eci_1_update[1,intorbit],eci_1_update[2,intorbit],eci_1_update[3,intorbit]), 
                   Point3f(eci_2_update[1,intorbit],eci_2_update[2,intorbit],eci_2_update[3,intorbit]),
                   Point3f(eci_3_update[1,intorbit],eci_3_update[2,intorbit],eci_3_update[3,intorbit]),
                   Point3f(eci_4_update[1,intorbit],eci_4_update[2,intorbit],eci_4_update[3,intorbit])]
    
    
    #Find initial guess for TOA
    centroid_guess = [(eci_1_update[1,intorbit]+eci_2_update[1,intorbit]+eci_3_update[1,intorbit]+eci_4_update[1,intorbit])/4,
                      (eci_1_update[2,intorbit]+eci_2_update[2,intorbit]+eci_3_update[2,intorbit]+eci_4_update[2,intorbit])/4,
                      (eci_1_update[3,intorbit]+eci_2_update[3,intorbit]+eci_3_update[3,intorbit]+eci_4_update[3,intorbit])/4] 
    
    onearth = sECEFtoGEOC(centroid_guess, use_degrees = true)
    geodetic = [onearth[1], onearth[2], 0]

    #Guess
    xyz = sGEOCtoECEF(geodetic, use_degrees = true)*1e-3
    
    #sat poses is in km. need to switch to meters to find elev angle
    elev_angles = elevation_angle(sat_poses*1e3, r0_tag_update[1:3]*1e3)
    
    #Check if the satellites are in the horizon
    
    inhorizon = all(x->x<70, elev_angles)
    if inhorizon==false
        println("Not in the horizon")
        #If the satellites are not in the horizon, do not plot anything
        ellipse_xy1_TOA[] = [0]
        ellipse_xy2_TOA[] = [0]
        
        ellipse_xz1_TOA[] = [0]
        ellipse_xz2_TOA[] = [0]
        
        ellipse_yz1_TOA[] = [0]
        ellipse_yz2_TOA[] = [0]
        
        ellipse_xy1_Doppler[] = [0]
        ellipse_xy2_Doppler[] = [0]
        
        ellipse_xz1_Doppler[] = [0]
        ellipse_xz2_Doppler[] = [0]
        
        ellipse_yz1_Doppler[] = [0]
        ellipse_yz2_Doppler[] = [0]
        
        PDOP_TOA_text[] = "N/A"
        PDOP_Doppler_text[] = "N/A"
        
    else 
        #Get the covariance of the satellites and measurments
        
        time_accuracy = time_acc*1e-9
        
        #time_acc = 1*10^time_acc_exp
        freq_accuracy = freq_acc*1e-3
        
        P_TOA, P_Doppler = get_P(time_accuracy, freq_accuracy)
                
        #calculate the TOA covariance
        
        f2_final = f1 + (f2_observable*10^6)
        
        PDOP_TOA, pose_covariance_TOA = calculate_covariance_TOA(sat_poses, elev_angles, xyz, P_TOA, r0_scaled_TOA_update, f2_final)
        
        if PDOP_TOA == 0
            
            PDOP_TOA_text[] = "ERROR"
            
        else
            
            PDOP_TOA_rounded = round(PDOP_TOA, digits=3)
            PDOP_TOA_text[] = string(PDOP_TOA_rounded)
        
            pose_covariance_result_TOA = pose_covariance_TOA
            
        end
        
        #calculate the Doppler covariance
        PDOP_Doppler, pose_covariance_Doppler = calculate_covariance_Doppler(pose1, pose2, pose3, pose4, zenith_angles, zdot, P_Doppler, r0_scaled_Doppler_update, f2_final)
        
        if PDOP_Doppler == 0
            
            PDOP_Doppler_text[] = "ERROR"
            
        else
            
            PDOP_Doppler_rounded = round(PDOP_Doppler, digits=3)
            PDOP_Doppler_text[] = string(PDOP_Doppler_rounded)

            pose_covariance_result_Doppler = pose_covariance_Doppler
            
        end
        
        pose_covariance_xz_TOA = [pose_covariance_result_TOA[1,1] pose_covariance_result_TOA[1,3];pose_covariance_result_TOA[3,1] pose_covariance_result_TOA[3,3]]
        pose_covariance_xz_Doppler = [pose_covariance_result_Doppler[1,1] pose_covariance_result_Doppler[1,3];pose_covariance_result_Doppler[3,1] pose_covariance_result_Doppler[3,3]]
        
        #draw the covariance ellipses based off of the covariance matrix
        
        r_ellipse_xy_TOA = draw_ellipse(pose_covariance_result_TOA[1:2, 1:2])
        r_ellipse_xz_TOA = draw_ellipse(pose_covariance_xz_TOA)
        r_ellipse_yz_TOA = draw_ellipse(pose_covariance_result_TOA[2:3, 2:3])
        
        r_ellipse_xy_Doppler = draw_ellipse(pose_covariance_result_Doppler[1:2, 1:2])
        r_ellipse_xz_Doppler = draw_ellipse(pose_covariance_xz_Doppler)
        r_ellipse_yz_Doppler = draw_ellipse(pose_covariance_result_Doppler[2:3, 2:3])
        
        #update the ellipses observable
        ellipse_xy1_TOA[] = r_ellipse_xy_TOA[:,1]
        ellipse_xy2_TOA[] = r_ellipse_xy_TOA[:,2]
        
        ellipse_xz1_TOA[] = r_ellipse_xz_TOA[:,1]
        ellipse_xz2_TOA[] = r_ellipse_xz_TOA[:,2]
        
        ellipse_yz1_TOA[] = r_ellipse_yz_TOA[:,1]
        ellipse_yz2_TOA[] = r_ellipse_yz_TOA[:,2]
        
        ellipse_xy1_Doppler[] = r_ellipse_xy_Doppler[:,1]
        ellipse_xy2_Doppler[] = r_ellipse_xy_Doppler[:,2]
        
        ellipse_xz1_Doppler[] = r_ellipse_xz_Doppler[:,1]
        ellipse_xz2_Doppler[] = r_ellipse_xz_Doppler[:,2]
        
        ellipse_yz1_Doppler[] = r_ellipse_yz_Doppler[:,1]
        ellipse_yz2_Doppler[] = r_ellipse_yz_Doppler[:,2]
          
    end
    
    #update the guess observable
    guess[] = xyz
    end
    
#Initialize the sliders at these positions

set_close_to!(lsgrid.sliders[1], 1200)
set_close_to!(lsgrid.sliders[2], 2)
set_close_to!(lsgrid.sliders[3], 10)
set_close_to!(lsgrid.sliders[4], 3)
set_close_to!(lsgrid.sliders[5], 1)
set_close_to!(lsgrid.sliders[6], 10)
set_close_to!(lsgrid.sliders[7], 100)
set_close_to!(lsgrid.sliders[8], 0)
set_close_to!(lsgrid.sliders[9], 200)


#Plot tag position

tagpose = meshscatter!(tag_xyzt, markersize = 150, color=:red, label = "Tag")

#Plot the Guess
guesspose = meshscatter!(guess, markersize = 150, color=:orange, label= "Guess")

#Plot satellites
satpose = meshscatter!(sats, markersize = 150, color=:yellow, label= "Satellites")

#Plot orbit  of sat 1/2
orbit12 = lines!(eci_1_x, eci_1_y, eci_1_z, color = :blue, label= "Orbit 1/2")

#Plot orbit of sat 3/4
orbit34 = lines!(eci_3_x, eci_3_y, eci_3_z, color = :purple, label= "Orbit 3/4")


#Create axes for covariance plots and include all titles
ax1 = Axis(ellipse_fig[1, 1], xlabel = "East (m)", ylabel = "North (m)",
    title = "2 Sigma Confidence Bound")
ax2 = Axis(ellipse_fig[2, 1], xlabel = "East (m)", ylabel = "Up (m)",
    title = "2 Sigma Confidence Bound")
ax3 = Axis(ellipse_fig[1, 2], xlabel = "North (m)", ylabel = "Up (m)",
    title = "2 Sigma Confidence Bound")
ax4 = Axis(ellipse_fig[2, 2], title = "Pose Standard Deviation")

hidedecorations!(ax4)

#set the limits on the covariance plots
xlims!(ax1,-200, 200)
xlims!(ax2, -200, 200)
xlims!(ax3, -200, 200)

ylims!(ax1, -200, 200)
ylims!(ax2, -200, 200)
ylims!(ax3, -200, 200)

#Plot Covariance Ellipses
ellipse1_TOA= lines!(ax1, ellipse_xy1_TOA, ellipse_xy2_TOA, label="TOA")
ellipse2_TOA = lines!(ax2, ellipse_xz1_TOA, ellipse_xz2_TOA, label = "TOA")
ellipse3_TOA = lines!(ax3, ellipse_yz1_TOA, ellipse_yz2_TOA, label = "TOA")
#text!(7, 0.38, text = L"\frac{\sin(x)}{\sqrt{x}}", color = :black)

ellipse1_Doppler= lines!(ax1, ellipse_xy1_Doppler, ellipse_xy2_Doppler, label = "Doppler")
ellipse2_Doppler = lines!(ax2, ellipse_xz1_Doppler, ellipse_xz2_Doppler, label = "Doppler")
ellipse3_Doppler = lines!(ax3, ellipse_yz1_Doppler, ellipse_yz2_Doppler, label = "Doppler")

#text(ellipse_fig[2,2], PDOP_TOA_text, position=(-200, 0), align = (:center, :center))

text!(ax4, PDOP_TOA_text, position=(0, 0), align = (:left, :center), textsize = 15)
text!(ax4, PDOP_Doppler_text, position=(1, 0), align = (:right, :center), textsize = 15)
text!(ax4, "TOA", position=(0, -1), align = (:left, :bottom), textsize = 20)
text!(ax4, "Doppler", position=(1, -1), align = (:right, :bottom), textsize = 20)

#text!(PDOP_TOA_text, position=(-200, 0))

ellipse_fig[3, 1:2] = Legend(ellipse_fig, ax1, "Methods", framevisible = false)

display(fig)

┌ Warning: labelslidergrid! is deprecated and will be removed in the future. Use SliderGrid instead.
│   caller = ip:0x0
└ @ Core :-1


Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon


GLMakie.Screen(...)

Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the horizon
Not in the h